# 电力生产 1

## 目标和前提条件

世界各地的大型电力公司都利用数学优化来管理其电网中的能源流动。在这个例子中，您将发现数学优化在解决常见能源行业问题方面的优势：电力生产。我们将向您展示如何确定最佳的发电站组合，以满足24小时时间范围内的预期电力需求。

这个模型是H. Paul Williams所著《Model Building in Mathematical Programming》第五版第270-271页和325-326页中的示例15。

这是一个中级难度的示例，我们假设您了解Python和Gurobi Python API，并且对构建数学优化模型有一定了解。

**下载代码库** <br />
您可以通过点击[这里](https://github.com/Gurobi/modeling-examples/archive/master.zip)下载包含此示例和其他示例的代码库。

---
## 问题描述

在这个问题中，发电机组被分为三种不同类型，每种类型都有不同的特性（发电量、每兆瓦时成本、启动成本等）。一个机组可以开启或关闭，从关闭到开启会产生启动成本，当机组开启时，发电量可以在指定的最小值和最大值之间变化。24小时的时间范围被分为5个离散时间段，每个时段都有预期的总电力需求。模型需要决定在什么时候开启哪些机组，以满足每个时间段的需求。该模型还考虑了备用要求，即选定的发电厂必须能够在不超过其最大输出的情况下增加输出，以应对实际需求超过预测需求的情况。

有一组发电机可用于满足次日的电力需求。预期需求如下：

| 时间段 | 需求（兆瓦） |
| --- | --- |
| 凌晨12点至早上6点 | 15000 |
| 早上6点至9点 | 30000 |
| 早上9点至下午3点 | 25000 |
| 下午3点至6点 | 40000 |
| 晚上6点至凌晨12点 | 27000 |

发电机分为三种类型，每种类型在开启时的最小和最大输出如下：

| 类型 | 可用数量 | 最小输出(MW) | 最大输出(MW) |
| --- | --- | --- | --- |
| 0 | 12 | 850 | 2000 |
| 1 | 10 | 1250 | 1750 |
| 2 | 5 | 1500 | 4000 |

使用发电机会产生相关成本：发电机开启时（并产生最小输出）的每小时成本、超过最小输出的每兆瓦时成本，以及开启发电机的启动成本：

| 类型 | 每小时成本（开启状态） | 超出最小值的每兆瓦时成本 | 启动成本 |
| --- | --- | --- | --- |
| 0 | $\$1000$ | $\$2.00$ | $\$2000$ |
| 1 | $\$2600$ | $\$1.30$ | $\$1000$ |
| 2 | $\$3000$ | $\$3.00$ | $\$500$ |

发电机不仅要满足预测需求，还必须具有足够的备用容量，以应对实际需求超过预测需求的情况。在这个模型中，所选的发电机组必须能够产生预测需求的115%。

为了最小化总成本，应该选择哪些发电机来满足预期需求？

---
## 模型构建

### 集合和索引

$t \in \text{Types}=\{0,1,2\}$：发电机类型。

$p \in \text{Periods}=\{0,1,2,3,4\}$：时间段。

### 参数

$\text{period_hours}_p \in \mathbb{N}^+$：每个时间段的小时数。

$\text{generators}_t \in \mathbb{N}^+$：类型$t$的发电机数量。

$\text{demand}_p \in \mathbb{R}^+$：时间段$p$的总电力需求。

$\text{start0} \in \mathbb{N}^+$：时间范围开始时已开启的发电机数量（在时间段0可用且无需支付启动成本）。

$\text{min_output}_t \in \mathbb{R}^+$：类型$t$发电机的最小输出（开启状态）。

$\text{max_output}_t \in \mathbb{R}^+$：类型$t$发电机的最大输出。

$\text{base_cost}_t \in \mathbb{R}^+$：类型$t$发电机的最小运营成本（每小时）。

$\text{per_mw_cost}_t \in \mathbb{R}^+$：类型$t$发电机每增加一兆瓦的成本（每小时）。

$\text{startup_cost}_t \in \mathbb{R}^+$：类型$t$发电机的启动成本。

### 决策变量

$\text{ngen}_{t,p} \in \mathbb{N}^+$：时间段$p$中类型$t$的开启发电机数量。

$\text{output}_{t,p} \in \mathbb{R}^+$：时间段$p$中类型$t$发电机的总输出功率。

$\text{nstart}_{t,p} \in \mathbb{N}^+$：时间段$p$中需要启动的类型$t$发电机数量。


### 目标函数

- **成本**：最小化满足预测电力需求的成本（美元）。

\begin{equation}
\text{最小化} \quad Z_{on} + Z_{extra} + Z_{startup}
\end{equation}

\begin{equation}
Z_{on} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{base_cost}_t*\text{ngen}_{t,p}}
\end{equation}

\begin{equation}
Z_{extra} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{per_mw_cost}_t*(\text{output}_{t,p} - \text{min_load}_t})
\end{equation}

\begin{equation}
Z_{startup} = \sum_{(t,p) \in \text{Types} \times \text{Periods}}{\text{startup_cost}_t*\text{nstart}_{t,p}}
\end{equation}

### 约束条件

- **可用发电机**：使用的发电机数量必须小于或等于可用数量。

\begin{equation}
\text{ngen}_{t,p} \leq \text{generators}_{t} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **需求**：所有类型发电机的总发电量必须满足每个时间段$p$的预期需求。

\begin{equation}
\sum_{t \in \text{Types}}{\text{output}_{t,p}} \geq \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **最小/最大发电量**：发电量必须遵守发电机的最小/最大值。

\begin{equation}
\text{output}_{t,p} \geq \text{min_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

\begin{equation}
\text{output}_{t,p} \leq \text{max_output}_t*\text{ngen}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

- **备用**：选定的发电机必须能够满足比预测高出15%的需求。

\begin{equation}
\sum_{t \in \text{Types}}{\text{max_output}_t*\text{ngen}_{t,p}} \geq 1.15 * \text{demand}_p \quad \forall p \in \text{Periods}
\end{equation}

- **启动**：建立活跃发电机数量与启动数量之间的关系（使用$start0$作为时间范围开始前的时段）

\begin{equation}
\text{ngen}_{t,p} \leq \text{ngen}_{t,p-1} + \text{startup}_{t,p} \quad \forall (t,p) \in \text{Types} \times \text{Periods}
\end{equation}

---
## Python实现

我们导入Gurobi Python模块和其他Python库。

In [ ]:
# %pip install gurobipy

In [1]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.11 & Gurobi 11.0

## 输入数据
我们定义模型的所有输入数据。

In [2]:
# 参数定义

ntypes = 3
nperiods = 5
maxstart0 = 5

generators = [12, 10, 5]
period_hours = [6, 3, 6, 3, 6]
demand = [15000, 30000, 25000, 40000, 27000]
min_load = [850, 1250, 1500]
max_load = [2000, 1750, 4000]
base_cost = [1000, 2600, 3000]
per_mw_cost = [2, 1.3, 3]
startup_cost = [2000, 1000, 500]

## 模型部署

我们创建一个模型和变量。对于每个时间段，我们有：一个整数决策变量来捕获每种类型的活跃发电机数量（ngen），一个整数变量来捕获我们必须启动的每种类型发电机数量（nstart），以及一个连续决策变量来捕获每种发电机类型的总输出功率（output）。

In [3]:
model = gp.Model('PowerGeneration')
ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

Set parameter LicenseID to value 2601452


接下来我们插入约束条件：

活跃发电机的数量不能超过可用发电机的数量。

In [4]:
# 发电机数量约束

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))

发电机类型的总输出功率取决于该类型的活跃发电机数量。

In [5]:
# 遵守每种发电机类型的最小和最大输出限制

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

每个时间段的总输出必须满足预测需求。

In [6]:
# 满足需求

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) >= demand[period]
                               for period in range(nperiods))

选定的发电机必须能够应对额外的需求。

In [7]:
# 提供足够的备用容量

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >= 1.15*demand[period]
                    for period in range(nperiods))

连接捕获活跃发电机的决策变量与计算启动次数的决策变量。

In [8]:
# 启动约束

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))

目标：最小化总成本。成本由三部分组成：运行活跃发电机的成本、每个机组超过最小发电量的发电成本，以及启动发电机的成本。

In [9]:
# 目标：最小化总成本

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj)

接下来，我们开始优化，Gurobi将找到最优解。

In [10]:
# model.write('junk.lp')
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 70 rows, 45 columns and 147 nonzeros
Model fingerprint: 0xd010fb19
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]
Presolve removed 31 rows and 1 columns
Presolve time: 0.00s
Presolved: 39 rows, 44 columns, 115 nonzeros
Variable types: 0 continuous, 44 integer (0 binary)
Found heuristic solution: objective 1339300.0000

Root relaxation: objective 9.995143e+05, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9

---
## 分析

24小时时间窗口内的预期电力需求可以以总成本$\$1,002,540$满足。每个时间段的详细计划如下：

### 机组调度

下表显示了最优解中每个时间段内每种类型的活跃发电机数量：

In [11]:
rows = ["Type" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Type"+str(t), p] = ngen[t,p].x
units

,0,1,2,3,4
Type0,12.0,12.0,12.0,12.0,12.0
Type1,3.0,8.0,8.0,9.0,9.0
Type2,-0.0,0.0,0.0,2.0,-0.0


以下显示了在每个时间段需要启动的各类型发电机数量，以实现这个计划（请记住，模型假设在时间范围开始时有5台发电机可用）：

In [12]:
startups = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        startups.loc["Type"+str(t), p] = int(nstart[t,p].x)
startups

,0,1,2,3,4
Type0,7.0,0.0,0.0,0.0,0.0
Type1,0.0,5.0,0.0,1.0,0.0
Type2,0.0,0.0,0.0,2.0,0.0


---
## 参考文献

H. Paul Williams, 《Model Building in Mathematical Programming》第五版。

版权所有 © 2020 Gurobi Optimization, LLC